In [40]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import when, lit, count
import json
from pyspark.sql.types import IntegerType, StringType
from datetime import datetime
import boto3

#### Boto3 session to connect to S3

In [41]:
# accessKey = input()

In [42]:
# secretKey = input()

In [43]:
session = boto3.Session(
    aws_access_key_id=accessKey,
    aws_secret_access_key=secretKey,
)
s3 = session.resource('s3')

#### Func: Transform NDJSON to JSON

In [45]:
def ndjson_to_json(resource):
    with open('fhir/ndjson/'+ resource +'.ndjson') as f:
        for line in f:
            json_data = json.loads(line)
            yield json_data

#### Upload to S3

In [46]:
resources_for_s3 = ['Patient', 'Encounter', 'DiagnosticReport']

In [48]:
def s3_upload_resources(db_name, resources):
    for resource in resources:
        objs_to_upload = ndjson_to_json(resource)
        file_name = resource + '.json'
        s3object = s3.Object(db_name, file_name)
        for obj in objs_to_upload:
            s3object.put(
                Body=(bytes(json.dumps(obj).encode('UTF-8')))
            )

In [49]:
s3_upload_resources('mfey-fhir-dynamodb', resources_for_s3)